# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233369197404                   -0.50    7.0         
  2   -7.249546661578       -1.79       -1.39    1.0   9.78ms
  3   -7.251006445003       -2.84       -1.83    2.0   11.9ms
  4   -7.251033104674       -4.57       -1.92    1.0   10.6ms
  5   -7.251314016890       -3.55       -2.48    1.0   10.0ms
  6   -7.251338028843       -4.62       -3.06    2.0   12.1ms
  7   -7.251338654996       -6.20       -3.43    1.0   10.3ms
  8   -7.251338738351       -7.08       -3.67    2.0   12.6ms
  9   -7.251338793161       -7.26       -4.22    1.0   10.6ms
 10   -7.251338798386       -8.28       -4.92    2.0   69.9ms
 11   -7.251338798674       -9.54       -5.30    2.0   14.2ms
 12   -7.251338798699      -10.60       -5.63    2.0   13.1ms
 13   -7.251338798704      -11.28       -6.26    1.0   11.2ms
 14   -7.251338798704      -12.36       -6.53    2.0   12.9ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06655470344746517
[ Info: Arnoldi iteration step 2: normres = 0.5308443818861538
[ Info: Arnoldi iteration step 3: normres = 0.7324018766797052
[ Info: Arnoldi iteration step 4: normres = 0.21187680492240643
[ Info: Arnoldi iteration step 5: normres = 0.41516144924405196
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.34e-02, 3.45e-02, 3.61e-01, 2.02e-01, 1.23e-02)
[ Info: Arnoldi iteration step 6: normres = 0.368521000951862
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.52e-03, 2.64e-01, 2.05e-01, 1.16e-01, 6.89e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0880467345779853
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.97e-04, 2.29e-02, 5.83e-03, 3.99e-02, 6.82e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12425911082990389
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.07e-0